# Lista 3
Arthur Pontes de Miranda Ramos Soares

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

## Funções Auxiliares

In [ ]:
def show_images(*images: np.ndarray, titles: list[str] | None = None, columns: int = 2, scale: int = 5) -> None:
    num_images = len(images)
    
    if titles is None:
        titles = [f'Image {i+1}' for i in range(num_images)]
    
    rows = (num_images + columns - 1) // columns 

    fig, axes = plt.subplots(rows, columns, figsize=(scale * columns, scale * rows))
    axes = np.array(axes).reshape(rows, columns)

    for ax, img, title in zip(axes.flat, images, titles):
        ax.imshow(img)
        ax.set_title(title)

    for i in range(num_images, rows * columns):
        fig.delaxes(axes.flat[i])

    plt.tight_layout()

def show_image(image: np.ndarray, title: str = None, dpi: int = 100) -> None:
    height, width, _ = image.shape
    
    figsize = (width / dpi, height / dpi)
    
    plt.figure(figsize=figsize, dpi=dpi)
    plt.imshow(image, cmap='gray' if len(image.shape) == 2 else None)
    plt.title(title if title else "")

    plt.tight_layout()

## Questão 1

In [ ]:
N_MATCHES = 15
def match_sift(img1: cv.typing.MatLike, img2: cv.typing.MatLike):
    sift = cv.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    bf = cv.BFMatcher(cv.NORM_L2, crossCheck=True)

    matches = bf.match(des1, des2)
    matches = sorted(matches, key = lambda x:x.distance)
    
    return kp1, kp2, matches

# filenames = [('predios-esq', 'predios-dir')]
filenames = [('mesa-left', 'mesa-right')]
for filename1, filename2 in filenames:
    left = cv.imread(f'./assets/{filename1}.jpg', cv.IMREAD_GRAYSCALE)
    right = cv.imread(f'./assets/{filename2}.jpg', cv.IMREAD_GRAYSCALE)
    kp1, kp2, matches = match_sift(left, right)

    src = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 2)
    dst = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 2)
    print(src)
    print(dst)
    img3 = cv.drawMatches(left, kp1, right, kp2, matches[:N_MATCHES], None, 
                          flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
                          matchColor=[255, 0, 0])
    H, mask = cv.findHomography(src, dst, cv.RANSAC)
    print(H)
    
    panorama_width = left.shape[1] + right.shape[1]
    panorama_height = max(left.shape[0], right.shape[0])
    print(panorama_height, panorama_width)

    res = cv.warpPerspective(left, H, (panorama_width, panorama_height), flags=cv.INTER_LINEAR)
    # res[0:right.shape[0], 0:right.shape[1]] = right
    show_images(res, right, titles=['res', 'right'], scale=10)
    # show_images(img3, res, left, right, titles=['matches', 'blablabla', 'left', 'right'], scale=10)


In [ ]:
def create_panorama(img_left, img_right, N_MATCHES=15):
    """
    Cria um panorama a partir de duas imagens
    """
    # Converter para escala de cinza se as imagens forem coloridas
    if len(img_left.shape) == 3:
        img_left_gray = cv.cvtColor(img_left, cv.COLOR_BGR2GRAY)
        img_right_gray = cv.cvtColor(img_right, cv.COLOR_BGR2GRAY)
    else:
        img_left_gray = img_left.copy()
        img_right_gray = img_right.copy()
    
    # Encontrar correspondências entre as imagens
    kp1, kp2, matches = match_sift(img_left_gray, img_right_gray)
    
    # Obter os pontos correspondentes
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 2)
    
    # Desenhar as correspondências
    img_matches = cv.drawMatches(img_left, kp1, img_right, kp2, matches[:N_MATCHES], None,
                        flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
                        matchColor=[0, 255, 0])
    
    # Calcular a homografia
    H, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 5.0)
    
    # Calcular as dimensões do panorama
    h_left, w_left = img_left.shape[:2]
    h_right, w_right = img_right.shape[:2]
    
    # Determinar os cantos da imagem da esquerda após a transformação
    corners_left = np.float32([[0, 0], [0, h_left], [w_left, h_left], [w_left, 0]]).reshape(-1, 1, 2)
    corners_left_transformed = cv.perspectiveTransform(corners_left, H)
    
    # Encontrar os valores mínimos e máximos após a transformação
    [x_min, y_min] = np.int32(corners_left_transformed.min(axis=0).ravel())
    [x_max, y_max] = np.int32(corners_left_transformed.max(axis=0).ravel())
    
    # Ajustar a matriz de transformação para lidar com deslocamentos
    translation_dist = [-x_min, -y_min]
    H_translation = np.array([[1, 0, translation_dist[0]], [0, 1, translation_dist[1]], [0, 0, 1]])
    H_adjusted = H_translation @ H
    
    # Calcular as dimensões finais do panorama
    panorama_width = max(x_max - x_min, w_right) + translation_dist[0]
    panorama_height = max(y_max - y_min, h_right) + translation_dist[1]
    
    # Criar o panorama
    panorama = cv.warpPerspective(img_left, H_adjusted, (panorama_width, panorama_height))
    
    # Adicionar a imagem da direita ao panorama
    x_offset = translation_dist[0]
    y_offset = translation_dist[1]
    panorama[y_offset:y_offset+h_right, x_offset:x_offset+w_right] = img_right
    
    return panorama, img_matches

In [ ]:
filenames = [('sala-left', 'sala-right')]
for filename1, filename2 in filenames:
    left = cv.imread(f'./assets/{filename1}.jpg', cv.IMREAD_GRAYSCALE)
    right = cv.imread(f'./assets/{filename2}.jpg', cv.IMREAD_GRAYSCALE)
    pan, matches = create_panorama(left, right, 15)
    show_images(pan, matches, titles=['pan', 'matches'], scale=10)

In [ ]:
import cv2 as cv
import numpy as np

N_MATCHES = 200

def match_sift(img1, img2):
    sift = cv.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    bf = cv.BFMatcher(cv.NORM_L2, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)
    return kp1, kp2, matches


left = cv.imread('./assets/gym-left.jpg', cv.IMREAD_GRAYSCALE)
right = cv.imread('./assets/gym-right.jpg', cv.IMREAD_GRAYSCALE)

kp1, kp2, matches = match_sift(left, right)

# Pegue muitos matches!
src_pts = np.float32([kp1[m.queryIdx].pt for m in matches[:N_MATCHES]]).reshape(-1, 1, 2)
dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches[:N_MATCHES]]).reshape(-1, 1, 2)

H, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC)

# Descobrir o tamanho necessário do panorama
h1, w1 = left.shape
h2, w2 = right.shape

# Coordenadas dos cantos da imagem da esquerda
pts_left = np.float32([[0, 0], [0, h1], [w1, h1], [w1, 0]]).reshape(-1, 1, 2)
pts_left_transformed = cv.perspectiveTransform(pts_left, H)

# Juntando com os cantos da imagem da direita
pts_right = np.float32([[0, 0], [0, h2], [w2, h2], [w2, 0]]).reshape(-1, 1, 2)
all_pts = np.concatenate((pts_left_transformed, pts_right), axis=0)

[x_min, y_min] = np.int32(all_pts.min(axis=0).ravel() - 0.5)
[x_max, y_max] = np.int32(all_pts.max(axis=0).ravel() + 0.5)

# Translação pra garantir tudo positivo
translation = [-x_min, -y_min]
T = np.array([[1, 0, translation[0]],
              [0, 1, translation[1]],
              [0, 0, 1]])

# Warp da imagem esquerda
result = cv.warpPerspective(left, T @ H, (x_max - x_min, y_max - y_min))

# Colar a imagem da direita
result[translation[1]:h2 + translation[1], translation[0]:w2 + translation[0]] = right


show_images(result, titles=['res'], scale=10)